In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
#Parametros de ejecucion del programa
nombreArchivoEstructuraDataset = "./Mapas de Transformación de Datos/Datos en Pasos Intermedios/Estructura del Dataset.txt"
nombreArchivoDatasetSinEtiquetar = "./Dataset sin Etiquetar.csv"
nombreArchivoDatasetEtiquetado = "./Dataset Etiquetado.csv"
nombreColumnaDuplicadaSalida = "Interested in buying a mobile home policy".replace(" ", "_")
nombreColumnaSalida = "Number of mobile home policies".replace(" ", "_")
nombreColumnaFase = "Phase"
etiquetaDatosEntrenamiento = "Training"
etiquetaDatosPrueba = "Test"
etiquetaVariablesCategoricas = "Categorical"

colorTextoVerde = (35, 155, 86)
colorTextoRojo = (176, 58, 46)

In [3]:
def printColouredText(text, foregroundColor = (0, 0, 0), backgroundColor = (255, 255, 255)):
    return "\033[48;2;{};{};{}m\033[38;2;{};{};{}m{}\033[38;2;0;0;0m\033[48;2;255;255;255m".format(backgroundColor[0], backgroundColor[1], backgroundColor[2], foregroundColor[0], foregroundColor[1], foregroundColor[2], text)

def df_style(val):
    return "font-weight: bold"

def buildCountingPivotTable(dataframeSource, valuesColumn, indexColumn, mainColumnToGroup, secondaryColumnToGroup):
    pivotTableWith2DetailLevels = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = [secondaryColumnToGroup, mainColumnToGroup], aggfunc = 'count')
    pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = mainColumnToGroup, aggfunc = 'count')
    pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
    pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
    
    
    
    fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)
    
    suma = fullPivotTable.sum()
    suma.name = "General Total"
    pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = pd.Index([suma.name], name = indexColumn))
    pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
    
    
    
    fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])
    
    for column in fullPivotTable.columns:
        fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]
    
    
    last_row1 = pd.IndexSlice[fullPivotTable.index[fullPivotTable.index != ""], fullPivotTable.columns[-3:]]
    fullPivotTable = fullPivotTable.style.applymap(df_style, subset = last_row1)
    
    
    #last_row2 = pd.IndexSlice[fullPivotTable.index[fullPivotTable.index != ""], ("", "Yes")]
    #last_row3 = pd.IndexSlice[fullPivotTable.index[fullPivotTable.index != ""], ("", suma.name)]
    #fullPivotTable = fullPivotTable.style.applymap(df_style, subset = last_row1)
    #fullPivotTable = fullPivotTable.style.applymap(df_style, subset = last_row2)
    #fullPivotTable = fullPivotTable.style.applymap(df_style, subset = last_row3)
    
    #summary = pd.DataFrame([[0, 0, 0, 0, 0], [1620, 203, 392, 651, 2236]], index=["None", "Total"])

    # get a handle on the row that starts with `"Total"`, i.e., the last row here
    #last_row = pd.IndexSlice[summary.index[summary.index == "Total"], :]
    # and apply styling to it via the `subset` arg; first arg is styler function above
    #fullPivotTable = fullPivotTable.style.applymap(df_style, subset = fullPivotTable[("", suma.name)])

    #display(summaryStyled)
    
    #fullPivotTable[("", suma.name)] = "<b>" + fullPivotTable[("", suma.name)].astype(str) + "</b>"
    #df['important_column'] = [f'<b>{x}</b>' for x in df['important_column']]
    #return pivotTableWith1DetailLevel
    return fullPivotTable

In [4]:
structureDatasetFile = pd.read_csv(nombreArchivoEstructuraDataset, sep = ";", decimal = ".")
fieldNamesList = structureDatasetFile["Field Name"].tolist()
respectiveVariableKindsList = structureDatasetFile["Variable Kind"].tolist()

In [5]:
notLabeledDatasetFile = pd.read_csv(nombreArchivoDatasetSinEtiquetar, sep = ";", decimal = ".")

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDatasetSinEtiquetar, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(notLabeledDatasetFile.index)) + " dato(s)", colorTextoVerde))

Se ha extraído la información del archivo ./Dataset sin Etiquetar.csv
Se tiene(n) 9822 dato(s)


In [6]:
notLabeledDatasetFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 87 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Customer Subtype                                  9822 non-null   int64 
 1   Number of houses                                  9822 non-null   int64 
 2   Avg size househol                                 9822 non-null   int64 
 3   Avg age                                           9822 non-null   int64 
 4   Customer main type                                9822 non-null   int64 
 5   Roman catholic                                    9822 non-null   int64 
 6   Protestant                                        9822 non-null   int64 
 7   Other religion                                    9822 non-null   int64 
 8   No religion                                       9822 non-null   int64 
 9   Married                       

In [7]:
labeledDatasetFile = pd.read_csv(nombreArchivoDatasetEtiquetado, sep = ";", decimal = ".")
labeledDatasetFile[nombreColumnaDuplicadaSalida] = labeledDatasetFile[nombreColumnaSalida].tolist()
labeledDatasetFile.replace({nombreColumnaDuplicadaSalida: {0: "No", 1: "Yes"}}, inplace = True)

#Filtrado de informacion para separar el dataset de entrenamiento y el dataset de prueba del dataset completo
trainingDatasetFile = labeledDatasetFile[labeledDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosEntrenamiento)]
testDatasetFile = labeledDatasetFile[labeledDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosPrueba)]

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDatasetEtiquetado, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(labeledDatasetFile.index)) + " dato(s)", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(trainingDatasetFile.index)) + " dato(s) de entrenamiento", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(testDatasetFile.index)) + " dato(s) de prueba", colorTextoVerde))

Se ha extraído la información del archivo ./Dataset Etiquetado.csv
Se tiene(n) 9822 dato(s)
Se tiene(n) de 5822 dato(s) de entrenamiento
Se tiene(n) de 4000 dato(s) de prueba


In [8]:
pivotTablesList = {}

prefix = "Label_"
for nombreColumna in notLabeledDatasetFile.columns:
    if nombreColumna != nombreColumnaSalida.replace("_", " ").replace("-", " ") and nombreColumna != nombreColumnaDuplicadaSalida.replace("_", " ").replace("-", " ") and nombreColumna != nombreColumnaFase:
        indexCol = fieldNamesList.index(nombreColumna)
        variableKind = respectiveVariableKindsList[indexCol]
        
        if variableKind == etiquetaVariablesCategoricas:
            print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, \"" + prefix + nombreColumna.replace(" ", "_").replace("-", "_") + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
            print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")
        else:
            print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, \"" + nombreColumna.replace(" ", "_").replace("-", "_") + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
            print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")
        #pivotTablesList[nombreColumna] = buildCountingPivotTable(notLabeledDatasetFile, nombreColumnaSalida, nombreColumna, nombreColumnaDuplicadaSalida, nombreColumnaFase)
        

#pivotTablesList[] = buildCountingPivotTable(notLabeledDatasetFile, nombreColumnaSalida, col, nombreColumnaDuplicadaSalida, nombreColumnaFase)

#display(pivotTable1)

pivotTablesList["Customer Subtype"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

pivotTablesList["Number of houses"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

pivotTablesList["Avg size househol"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Avg_size_househol", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size househol"])

pivotTablesList["Avg age"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Avg_age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

pivotTablesList["Customer main type"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_main_type", nombreC

In [9]:
pivotTablesList["Customer Subtype"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

In [10]:
summary = pd.DataFrame([[0, 0, 0, 0, 0], [1620, 203, 392, 651, 2236]], index=["None", "Total"])

# get a handle on the row that starts with `"Total"`, i.e., the last row here
last_row = pd.IndexSlice[summary.index[summary.index != ""], :]
#last_row = pd.IndexSlice[summary.index[summary.column == "4"], :]
# and apply styling to it via the `subset` arg; first arg is styler function above
summaryStyled = summary.style.applymap(df_style, subset=last_row)

display(summaryStyled)

,0,1,2,3,4
None,0,0,0,0,0
Total,1620,203,392,651,2236


In [11]:
pivotTablesList["Number of houses"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

In [12]:
pivotTablesList["Avg size househol"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Avg_size_househol", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size househol"])

In [13]:
pivotTablesList["Avg age"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Avg_age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

In [14]:
pivotTablesList["Customer main type"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Customer_main_type", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer main type"])

In [15]:
pivotTablesList["Roman catholic"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Roman_catholic", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Roman catholic"])

In [16]:
pivotTablesList["Protestant"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Protestant", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Protestant"])

In [17]:
pivotTablesList["Other religion"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Other_religion", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Other religion"])

In [18]:
pivotTablesList["No religion"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "No_religion", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["No religion"])

In [19]:
pivotTablesList["Married"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Married", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Married"])

In [20]:
pivotTablesList["Living together"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Living_together", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Living together"])

In [21]:
pivotTablesList["Other relation"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Other_relation", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Other relation"])

In [22]:
pivotTablesList["Singles"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Singles", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Singles"])

In [23]:
pivotTablesList["Household without children"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Household_without_children", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Household without children"])

In [24]:
pivotTablesList["Household with children"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Household_with_children", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Household with children"])

In [25]:
pivotTablesList["High level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "High_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["High level education"])

In [26]:
pivotTablesList["Medium level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Medium_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Medium level education"])

In [27]:
pivotTablesList["Lower level education"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Lower_level_education", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Lower level education"])

In [28]:
pivotTablesList["High status"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "High_status", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["High status"])

In [29]:
pivotTablesList["Entrepreneur"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Entrepreneur", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Entrepreneur"])

In [30]:
pivotTablesList["Farmer"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Farmer", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Farmer"])

In [31]:
pivotTablesList["Middle management"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Middle_management", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Middle management"])

In [32]:
pivotTablesList["Skilled labourers"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Skilled_labourers", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Skilled labourers"])

In [33]:
pivotTablesList["Unskilled labourers"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Unskilled_labourers", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Unskilled labourers"])

In [34]:
pivotTablesList["Social class A"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_A", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class A"])

In [35]:
pivotTablesList["Social class B1"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_B1", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class B1"])

In [36]:
pivotTablesList["Social class B2"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_B2", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class B2"])

In [37]:
pivotTablesList["Social class C"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_C", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class C"])

In [38]:
pivotTablesList["Social class D"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Social_class_D", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Social class D"])

In [39]:
pivotTablesList["Rented house"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Rented_house", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Rented house"])

In [40]:
pivotTablesList["Home owners"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Home_owners", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Home owners"])

In [41]:
pivotTablesList["1 car"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "1_car", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["1 car"])

In [42]:
pivotTablesList["2 cars"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "2_cars", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["2 cars"])

In [43]:
pivotTablesList["No car"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "No_car", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["No car"])

In [44]:
pivotTablesList["National Health Service"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "National_Health_Service", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["National Health Service"])

In [45]:
pivotTablesList["Private health insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Private_health_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Private health insurance"])

In [46]:
pivotTablesList["Income <30.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_<30.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income <30.000"])

In [47]:
pivotTablesList["Income 30-45.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_30_45.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 30-45.000"])

In [48]:
pivotTablesList["Income 45-75.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_45_75.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 45-75.000"])

In [49]:
pivotTablesList["Income 75-122.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_75_122.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 75-122.000"])

In [50]:
pivotTablesList["Income >123.000"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Income_>123.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income >123.000"])

In [51]:
pivotTablesList["Average income"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Average_income", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Average income"])

In [52]:
pivotTablesList["Purchasing power class"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Purchasing_power_class", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Purchasing power class"])

In [53]:
pivotTablesList["Contribution private third party insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_private_third_party_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution private third party insurance"])

In [54]:
pivotTablesList["Contribution third party insurance (firms)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_third_party_insurance_(firms)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution third party insurance (firms)"])

In [55]:
pivotTablesList["Contribution third party insurane (agriculture)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_third_party_insurane_(agriculture)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution third party insurane (agriculture)"])

In [56]:
pivotTablesList["Contribution car policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_car_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution car policies"])

In [57]:
pivotTablesList["Contribution delivery van policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_delivery_van_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution delivery van policies"])

In [58]:
pivotTablesList["Contribution motorcycle/scooter policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_motorcycle/scooter_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution motorcycle/scooter policies"])

In [59]:
pivotTablesList["Contribution lorry policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_lorry_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution lorry policies"])

In [60]:
pivotTablesList["Contribution trailer policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_trailer_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution trailer policies"])

In [61]:
pivotTablesList["Contribution tractor policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_tractor_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution tractor policies"])

In [62]:
pivotTablesList["Contribution agricultural machines policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Contribution_agricultural_machines_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution agricultural machines policies"])

In [63]:
pivotTablesList["Contribution moped policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_moped_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution moped policies"])

In [64]:
pivotTablesList["Contribution life insurances"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_life_insurances", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution life insurances"])

In [65]:
pivotTablesList["Contribution private accident insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_private_accident_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution private accident insurance policies"])

In [66]:
pivotTablesList["Contribution family accidents insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_family_accidents_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution family accidents insurance policies"])

In [67]:
pivotTablesList["Contribution disability insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_disability_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution disability insurance policies"])

In [68]:
pivotTablesList["Contribution fire policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_fire_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution fire policies"])

In [69]:
pivotTablesList["Contribution surfboard policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_surfboard_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution surfboard policies"])

In [70]:
pivotTablesList["Contribution boat policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_boat_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution boat policies"])

In [71]:
pivotTablesList["Contribution bicycle policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_bicycle_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution bicycle policies"])

In [72]:
pivotTablesList["Contribution property insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_property_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution property insurance policies"])

In [73]:
pivotTablesList["Contribution social security insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Label_Contribution_social_security_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution social security insurance policies"])

In [74]:
pivotTablesList["Number of private third party insurance"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_private_third_party_insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of private third party insurance"])

In [75]:
pivotTablesList["Number of third party insurance (firms)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_third_party_insurance_(firms)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of third party insurance (firms)"])

In [76]:
pivotTablesList["Number of third party insurance (agriculture)"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_third_party_insurance_(agriculture)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of third party insurance (agriculture)"])

In [77]:
pivotTablesList["Number of car policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_car_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of car policies"])

In [78]:
pivotTablesList["Number of delivery van policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_delivery_van_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of delivery van policies"])

In [79]:
pivotTablesList["Number of motorcycle/scooter policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_motorcycle/scooter_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of motorcycle/scooter policies"])

In [80]:
pivotTablesList["Number of lorry policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_lorry_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of lorry policies"])

In [81]:
pivotTablesList["Number of trailer policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_trailer_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of trailer policies"])

In [82]:
pivotTablesList["Number of tractor policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_tractor_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of tractor policies"])

In [83]:
pivotTablesList["Number of agricultural machines policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_agricultural_machines_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of agricultural machines policies"])

In [84]:
pivotTablesList["Number of moped policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_moped_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of moped policies"])

In [85]:
pivotTablesList["Number of life insurances"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_life_insurances", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of life insurances"])

In [86]:
pivotTablesList["Number of private accident insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_private_accident_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of private accident insurance policies"])

In [87]:
pivotTablesList["Number of family accidents insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_family_accidents_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of family accidents insurance policies"])

In [88]:
pivotTablesList["Number of disability insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_disability_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of disability insurance policies"])

In [89]:
pivotTablesList["Number of fire policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_fire_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of fire policies"])

In [90]:
pivotTablesList["Number of surfboard policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_surfboard_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of surfboard policies"])

In [91]:
pivotTablesList["Number of boat policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_boat_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of boat policies"])

In [92]:
pivotTablesList["Number of bicycle policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_bicycle_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of bicycle policies"])

In [93]:
pivotTablesList["Number of property insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_property_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of property insurance policies"])

In [94]:
pivotTablesList["Number of social security insurance policies"] = buildCountingPivotTable(labeledDatasetFile, nombreColumnaSalida, "Number_of_social_security_insurance_policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of social security insurance policies"])